# Convolutional Classifier

## CNN

Convolutional neural network

Networks which have convolutional layers

## Image classification

We try to classify a given image.

## Common structure

Image --> Base_network --> Head_network --> Result

* Base_network: extracts features
* Head_network: uses those features to classify

Base_networks are mostly pretrained.

We mostly add a not trained Head_network to a pretrained Base_network to get our results.

## Transfer learning

Reusing a pretrained network.

## ImageNet

A large scale dataset which so many models have trained on.

# Simple code based on kaggle course

## Import packages

In [2]:
import tensorflow as tf
from tensorflow import keras

from keras.applications.inception_v3 import InceptionV3
from keras import layers
from keras import Sequential

from pathlib import Path

image_dataset_from_directory = tf.keras.preprocessing.image_dataset_from_directory


## check if gpu is available

In [4]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Load Data

Data is downloaded from Kaggle.

link: [Kaggle](https://www.kaggle.com/datasets/ryanholbrook/car-or-truck)

In [6]:
data_source_path = Path("/home/ramin/ramin_programs/files/data/car_truck/")

data_train_path = data_source_path / "train"
data_valid_path = data_source_path / "valid"

data_train = image_dataset_from_directory(
    data_train_path,
    labels="inferred",
    label_mode="binary",
    image_size=[128, 128],
    interpolation="nearest",
    batch_size=64,
    shuffle=True,
)

data_valid = image_dataset_from_directory(
    data_valid_path,
    labels="inferred",
    label_mode="binary",
    image_size=[128, 128],
    interpolation="nearest",
    batch_size=64,
    shuffle=False,
)

Found 5117 files belonging to 2 classes.
Found 5051 files belonging to 2 classes.


In [7]:
# convert data to float
def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE
data_train = (
    data_train
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)
data_valid = (
    data_valid
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)

## defining base_model

In [10]:
base_model = tf.keras.models.load_model('/home/ramin/ramin_programs/files/models/cv-course-models/inceptionv1')
base_model.trainable = False

## Making our model

In [11]:
model = Sequential(
    [
        base_model,
        layers.Flatten(),
        layers.Dense(6, activation="relu"),
        layers.Dense(1, activation="sigmoid"),
    ]
)

In [12]:
optimizer = tf.keras.optimizers.Adam(epsilon=0.01)
model.compile(
    optimizer=optimizer,
    loss = 'binary_crossentropy',
    metrics=['binary_accuracy'],
)

In [ ]:
history = model.fit(
    data_train,
    validation_data=data_valid,
    epochs=30,
)